In [3]:
#PYTHON DATA PROJECT

#We begin by importing the relevant packages matplotlib, pandas and pydst. 

#IMPORT PACKAGES  
import matplotlib.pyplot as plt
import pandas as pd
import pydst


In [4]:
#Show all columns in iPython console regardless of amount.
pd.set_option("display.max_columns", None)

dst = pydst.Dst(lang='en')

#Import employment data from Statistics Denmark, specifically table RAS201 - All variables in the table are imported to get the full data set. 
Emp = dst.get_data(table_id = 'RAS201', variables={'OMRÅDE':['*'], 'TID':['*'], 'ALDER':['*'], 'HERKOMST':['*'], 'KØN':['*'], 'SOCIO': ['*']})


#Delete the columns age, gender and ancestry. Rename the remaining variables.
for v in ['ALDER', 'KØN', 'HERKOMST']: 
    del Emp[v]
Emp = Emp.rename(columns = {'TID':'Year', 'OMRÅDE': 'Region', 'SOCIO': 'Status', 'INDHOLD': 'Numpers'})



ConnectionError: HTTPSConnectionPool(host='api.statbank.dk', port=443): Max retries exceeded with url: /v1/tableinfo/RAS201?lang=en&format=JSON (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x000002CBCEA0DCC0>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))

In [ ]:
#Delete all municipalities such that only regions and total for Denmark is left. 
#We use a Regex code for logics to remove variables that does not contain 'Region' or 'All Denmark'. 
I = Emp['Region'].str.contains(r'^(?:(?!Region|All Denmark).)*$') 
Emp.drop(Emp[I].index, inplace=True)    
print(Emp)

#Aggregate data to region per year for different socio economical statuses. 
Agg_Emp=Emp.groupby(['Region', 'Year', 'Status'])['Numpers'].sum().reset_index()
Agg_Emp.head()



In [5]:
# Create line chart for Regions and All Denmark
TOT=Agg_Emp.groupby(['Region', 'Year'])['Numpers'].sum()

Status_Emp = Agg_Emp['Status']=='Employed'
Status_Emp.head()

Agg_Emp_New=Agg_Emp[Status_Emp]
Agg_Emp_New.head()

TOT_Emp=Agg_Emp_New.groupby(['Region', 'Year']).sum()
TOT_Emp.head()

Joined = pd.merge(TOT_Emp.reset_index(),TOT.reset_index(),how='inner',on=['Region','Year'])
print(Joined)

Joined = Joined.rename(columns = {'Numpers_x':'Employed', 'Numpers_y':'Population'})

Joined['Employ_share'] = Joined.Employed/Joined.Population*100
Joined.head()


                Region  Year  Numpers_x  Numpers_y
0          All Denmark  2008    2844814    5506684
1          All Denmark  2009    2725724    5530627
2          All Denmark  2010    2704542    5553663
3          All Denmark  2011    2698418    5575097
4          All Denmark  2012    2684840    5596882
5          All Denmark  2013    2693948    5619260
6          All Denmark  2014    2719840    5654401
7          All Denmark  2015    2759409    5699958
8          All Denmark  2016    2805073    5741655
9          All Denmark  2017    2848333    5775188
10  Region Hovedstaden  2008     870937    1660723
11  Region Hovedstaden  2009     848342    1678718
12  Region Hovedstaden  2010     846676    1694635
13  Region Hovedstaden  2011     852815    1711671
14  Region Hovedstaden  2012     856717    1728983
15  Region Hovedstaden  2013     866553    1746074
16  Region Hovedstaden  2014     878625    1765983
17  Region Hovedstaden  2015     898268    1786518
18  Region Hovedstaden  2016   

,Region,Year,Employed,Population,Employ_share
0,All Denmark,2008,2844814,5506684,51.661109
1,All Denmark,2009,2725724,5530627,49.284177
2,All Denmark,2010,2704542,5553663,48.698346
3,All Denmark,2011,2698418,5575097,48.401274
4,All Denmark,2012,2684840,5596882,47.970281
